#### Import arcgis and connect to ArcGIS Enterprise

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.geoanalytics.manage_data import run_python_script

gis = GIS("https://demo.ga.geocloud.com/portal", username="demo73")
arcgis.env.verbose=True

Enter password: ········


#### Define function to run on GeoAnalytics Server

In [2]:
def create_dead_tree_density_map():
    import uuid

    # Load tree point locations
    NYC_tree_url = "https://demo-gax.ga.geocloud.com/server/rest/services/DataStoreCatalogs/bigDataFileShares_ga-server-demo-data-bdfs/BigDataCatalogServer/NYC_tree_survey"
    NYC_tree_survey = spark.read.format("webgis").load(NYC_tree_url)

    # Load NYC borough polygons
    boroughs_url = "https://demo-gax.ga.geocloud.com/server/rest/services/DataStoreCatalogs/bigDataFileShares_ga-server-demo-data-bdfs/BigDataCatalogServer/NY_Borough_Boundaries"
    boroughs = spark.read.format("webgis").load(boroughs_url)

    # Project input data to same spatial reference
    NYC_tree_survey_proj = geoanalytics.project(input_features=NYC_tree_survey, output_coord_system=2263)
    boroughs_proj = geoanalytics.project(input_features=boroughs, output_coord_system=2263)

    # Clip tree data to Manhattan boundary
    trees_clipped = geoanalytics.clip_layer(NYC_tree_survey_proj, boroughs_proj.filter("boro_name = 'Manhattan'"))

    # Calculate density of dead trees
    tree_density = geoanalytics.calculate_density(trees_clipped.filter("status != 'Alive'"),
                                                  bin_type="Hexagon", bin_size=0.25, bin_size_unit="Miles",
                                                  radius=0.5, radius_unit="Miles",
                                                  area_units="SquareMiles")

    # Clip density bins to Manhattan boundary
    tree_density_clipped = geoanalytics.clip_layer(tree_density, boroughs_proj.filter("boro_name = 'Manhattan'"))

    # Create unique output name
    unique_id = str(uuid.uuid4())[:4]
    output_name = "manhattan_tree_density_" + unique_id

    tree_density_clipped.write.format("webgis").save(output_name)

#### Pass function to the Run Python Script tool

In [3]:
rps = run_python_script(create_dead_tree_density_map)

Submitted.
Executing...
Start Time: Wednesday, March 9, 2022 10:33:10 PM
{"messageCode":"BD_101028","message":"Starting new distributed job with 16 tasks.","params":{"totalTasks":"16"}}
{"messageCode":"BD_101029","message":"0/16 distributed tasks completed.","params":{"completedTasks":"0","totalTasks":"16"}}
{"messageCode":"BD_101029","message":"1/16 distributed tasks completed.","params":{"completedTasks":"1","totalTasks":"16"}}
{"messageCode":"BD_101029","message":"16/16 distributed tasks completed.","params":{"completedTasks":"16","totalTasks":"16"}}
{"messageCode":"BD_101028","message":"Starting new distributed job with 2 tasks.","params":{"totalTasks":"2"}}
{"messageCode":"BD_101029","message":"2/2 distributed tasks completed.","params":{"completedTasks":"2","totalTasks":"2"}}
{"messageCode":"BD_101028","message":"Starting new distributed job with 18 tasks.","params":{"totalTasks":"18"}}
{"messageCode":"BD_101029","message":"18/18 distributed tasks completed.","params":{"completed

#### Visualize result layer

In [ ]:
lyr_url = 'https://demo.ga.geocloud.com/server/rest/services/Hosted/manhattan_tree_density_a839/FeatureServer/0'

layer = arcgis.features.FeatureLayer(lyr_url)
tree_density_map = gis.map('Manhattan', 11)
tree_density_map.add_layer(layer, {"type": "FeatureLayer",
                                   "renderer":"ClassedColorRenderer",
                                   "field_name":"density",
                                   "opacity":0.75})
tree_density_map